In [17]:
def retrieve_documents():
    field1 = field1_entry.get()
    value1 = value1_entry.get()
    field2 = field2_entry.get()
    value2 = value2_entry.get()

    # Connect to the MongoDB server
    client = MongoClient('localhost', 27017)

    # Access the database and collection

    db = client['breast']
    collection = db['clinical_data']

    # Prepare the $and query
    query = {
        '$and': [
            {field1: value1},
            {field2: value2}
        ]
    }

    # Retrieve the documents matching the query
    documents = list(collection.find(query))

    # Create the GUI window
    window = tk.Tk()
    window.title('Clinical Data Results')
    screen_width = window.winfo_screenwidth()
    screen_height = window.winfo_screenheight()

# Set the window size to full screen
    window.geometry(f"{screen_width}x{screen_height}")

    # Create the treeview widget
    tree = ttk.Treeview(window)

    # Define the columns
    tree['columns'] = tuple(documents[0].keys()) if documents else ()

    # Format the columns
    for column in tree['columns']:
        tree.column(column, anchor='center')
        tree.heading(column, text=column)

    # Insert the document data into the treeview
    for doc in documents:
        tree.insert('', 'end', values=list(doc.values()))

    # Add scrollbar
    scrollbar = ttk.Scrollbar(window, orient="vertical", command=tree.yview)
    tree.configure(yscroll=scrollbar.set)
    scrollbar.pack(side="right", fill="y")

    # Pack the treeview
    tree.pack(expand=True, fill='both')

    # Run the GUI main loop
    window.mainloop()

# Create the GUI window
window = tk.Tk()
window.title('Retrieve Clinical Data')

# Create labels and text boxes for input
field1_label = tk.Label(window, text="Field 1:")
field1_label.pack()
field1_entry = tk.Entry(window)
field1_entry.pack()

value1_label = tk.Label(window, text="Value 1:")
value1_label.pack()
value1_entry = tk.Entry(window)
value1_entry.pack()

field2_label = tk.Label(window, text="Field 2:")
field2_label.pack()
field2_entry = tk.Entry(window)
field2_entry.pack()

value2_label = tk.Label(window, text="Value 2:")
value2_label.pack()
value2_entry = tk.Entry(window)
value2_entry.pack()

# Create the retrieve button
retrieve_button = tk.Button(window, text="Retrieve", command=retrieve_documents)
retrieve_button.pack()

# Run the GUI main loop
window.mainloop()

In [18]:
import tkinter as tk
from tkinter import ttk
from pymongo import MongoClient

def get_indexes():
    # Connect to the MongoDB database
    client = MongoClient('mongodb://localhost:27017')
    db = client['breast']
    collection = db['clinical_data']

    # Get the indexes of the collection
    indexes = collection.index_information()

    # Clear existing data in the labels
    index_info_label.config(text="")
    
    # Insert index data into the label
    for index_name, index_info in indexes.items():
        index_info_label.config(text=index_info_label.cget("text") + f"{index_name}: {index_info}\n")

# Create a Tkinter window
root = tk.Tk()
root.title("Indexing App")

# Create a label to display index information
index_info_label = tk.Label(root, text="", justify="left", font=("Arial", 12))
index_info_label.pack(pady=10)

# Create a button to get the indexes
indexes_button = tk.Button(root, text="Get Indexes", command=get_indexes)
indexes_button.pack()

# Start the Tkinter event loop
root.mainloop()

In [ ]:


#map_reduce
import tkinter as tk
from pymongo import MongoClient

# Establish a connection to the MongoDB server
client = MongoClient('mongodb://localhost:27017/')
database = client['breast']
collection = database['clinical_data']

def perform_map_reduce():
    pipeline = [
        {
            '$group': {
                '_id': '$Gender',
                'count': {'$sum': 1}
            }
        }
    ]

    result = list(collection.aggregate(pipeline))

    output = []
    for doc in result:
        output.append(f"{doc['_id']}: {doc['count']}")

    # Display the output in a Tkinter window
    window = tk.Tk()
    for i, line in enumerate(output):
        label = tk.Label(window, text=line)
        label.pack()
    window.mainloop()

# Create a Tkinter window
window = tk.Tk()

# Add a button to trigger the MapReduce operation
button = tk.Button(window, text="Perform MapReduce", command=perform_map_reduce)
button.pack()

# Run the Tkinter application
window.mainloop()

In [20]:
#check performance
import time
from pymongo import MongoClient

# Connect to the MongoDB database
client = MongoClient('mongodb://localhost:27017')
db = client['breast']
collection = db['clinical_data']

# Enable profiling to capture query performance information
db.command('profile', 2)

# Execute the query without the index and measure the execution time
start_time = time.time()
result_without_index = collection.find({"field_to_index": "some_value"})
end_time = time.time()

execution_time_without_index = end_time - start_time

# Create the index on the desired field
collection.create_index([("field_to_index", 1)])

# Execute the query with the index and measure the execution time
start_time = time.time()
result_with_index = collection.find({"ER": "positive"})
end_time = time.time()

execution_time_with_index = end_time - start_time

# Disable profiling
db.command('profile', 0)

# Print the execution times
print(f"Execution time without index: {execution_time_without_index} seconds")
print(f"Execution time with index: {execution_time_with_index} seconds")



Execution time without index: 7.605552673339844e-05 seconds
Execution time with index: 6.4849853515625e-05 seconds
